In [39]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import torch
from torch import nn
from collections import OrderedDict
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt


# In[29]:


class CoDA_Regress(nn.Module):
    
    def __init__(self, input_dim, dimension, encoder_shape, decoder_shape):
        super(CoDA_Regress, self).__init__()
        
        #define regression layer
        self.linear = nn.Linear(dimension, 1)
        
        encoder_dict = OrderedDict()
        
        #first layer will be twice input size, since we are feeding in both c_kl and X 
        encoder_dict["layer0"] = nn.Linear(2 * input_dim, encoder_shape[0])

        for i in range(0,len(encoder_shape)-1):
            encoder_dict["layer"  + str(i)] = nn.Linear(encoder_shape[i], encoder_shape[i+1])
            encoder_dict["layer_ac"  + str(i)] = nn.ELU()
        encoder_dict["final_layer"] = nn.Linear(encoder_shape[-1], dimension)
        encoder_dict["final_ac"] = nn.ELU()


        self.encoder = nn.Sequential(encoder_dict)
        
        decoder_dict = OrderedDict()
        decoder_dict["layer0"] = nn.Linear(dimension, decoder_shape[0])

        for i in range(0,len(decoder_shape)-1):
            decoder_dict["layer"  + str(i)] = nn.Linear(decoder_shape[i], decoder_shape[i+1])
            decoder_dict["layer_ac"  + str(i)] = nn.ELU()

        #final layer will map back to input dim
        decoder_dict["final_layer"] = nn.Linear(decoder_shape[-1], input_dim)
        decoder_dict["final_ac"] = nn.ELU()

        self.decoder = nn.Sequential(decoder_dict)
        
        
    def forward(self,x):
        #run the encoding and store the low level representation as A
        x_ckl = torch.log(torch.clamp(check(x), 1e-8,1))
        
        #pass in both x and x_ckl as per paper
        self.A = self.encoder(torch.cat((x, x_ckl), 1))
        self.reconstruction = self.decoder(self.A)
        self.pred = self.linear(self.A)
        #return both the predicted target, and the reconstruction so both can be inputs to the combined loss
        return self.pred, self.reconstruction, x_ckl
        


# In[39]:


class Combined_Loss(torch.nn.Module):
    def __init__(self, lam):
        super(Combined_Loss,self).__init__()
        self.CoDA_Loss = CoDA_Loss()
        self.MSE = nn.MSELoss()
        self.lam = lam
        
    def forward(self,Y,X,y_hat,y):
        #X is original data, Y is CoDA reconstruction, y is targets, y_hat 
        #input needs to be normalised by g(x) (geometric mean) for X_hat
        #TODO centering matrix? Reduce mean? Mask for near zero values?  
        
        #extract reconstruction and original data from concatenation
        
        return self.MSE(y_hat, y) + self.lam * self.CoDA_Loss(Y,X)  

class CoDA_Loss(torch.nn.Module):
    
    def __init__(self):
        super(CoDA_Loss,self).__init__()
        
    def forward(self,Y,X):
        #X is original data, Y is CoDA reconstruction
        X_check = check(X)
        #print ("XXX", X_check)
        coda_loss = torch.sum(torch.exp(torch.clamp(Y,-30,30))) - torch.sum(X_check * Y)
        
        print("Y", Y)
        print("exp", torch.sum(torch.exp(torch.clamp(Y,-30,30))))
        print ("tra",  torch.sum(X_check * Y))
        
        print ("LOSS", coda_loss)
        return coda_loss

def check(X):
    #assume input is tensor so we can use the numpy() method
    assert type(X) == torch.Tensor
    gmean = torch.prod(X, 1) ** (1./X.shape[1])
    return torch.div(X.t(),  torch.clamp(gmean, min=1e-8)).t()


# In[60]:


data = pd.read_csv("Data 18. Compositions and total pebble counts of 92 glacial tills.csv")




# headers = data[1]
# features = data[0][:,co_feature_indices]
# targets = data[0][:,target_index]
    
# #normalise the compositional features. TODO anything extra to deal with non compositional features?
# 


features = data[data.columns[1:-1]]
targets = data[data.columns[-1]]

features = np.array([feat/sum(feat) for feat in features.values])

features = torch.FloatTensor(features)
targets = torch.FloatTensor(targets)






In [40]:
# In[40]:


#training code stub, read in data as X and targets as y
#TODO substitute this into model class, and set up API similar to original CoDA-PCA paper
X = features
y = targets.reshape(-1,1)
model = CoDA_Regress(X.shape[1], 2, [100,], [3,])
#define the combined loss with hyperparameter lambda
l = 1
loss_function = Combined_Loss(l)
optim = torch.optim.SGD(model.parameters(), lr = 0.0001)
    
for epoch in range(0,1000):
    pred, recon, A = model.forward(torch.FloatTensor(X))
    loss = loss_function(recon, torch.FloatTensor(X), pred, torch.FloatTensor(y))

    
    optim.zero_grad()
        
    loss.backward()
    optim.step()
        
    epoch += 1
    
    if (epoch % 100 == 0):
        print("epoch {}, loss {}".format(epoch, loss))

    if (epoch == 1):
        print("A",A)


Y tensor([[-0.4564, -0.6538, -0.8205, -0.5616],
        [-0.0022, -0.2445, -0.4682, -0.1327],
        [ 0.0182, -0.2256, -0.4487, -0.1120],
        [-0.3639, -0.5087, -0.7486, -0.5044],
        [-0.0270, -0.2350, -0.4829, -0.1704],
        [ 0.0748, -0.1667, -0.3898, -0.0539],
        [-0.4823, -0.6643, -0.8335, -0.5898],
        [-0.0273, -0.2384, -0.4840, -0.1695],
        [-0.0267, -0.2545, -0.4877, -0.1621],
        [-0.0073, -0.2321, -0.4684, -0.1447],
        [-0.0442, -0.2535, -0.4995, -0.1865],
        [-0.4916, -0.6672, -0.8380, -0.6001],
        [-0.0017, -0.2480, -0.4687, -0.1304],
        [-0.3661, -0.5095, -0.7499, -0.5068],
        [-0.6998, -0.8096, -0.9340, -0.7881],
        [-0.0398, -0.2460, -0.4945, -0.1834],
        [-0.4859, -0.6654, -0.8353, -0.5938],
        [ 0.0733, -0.1683, -0.3915, -0.0556],
        [-0.4638, -0.6565, -0.8242, -0.5698],
        [-0.0150, -0.2313, -0.4736, -0.1555],
        [ 0.0760, -0.1647, -0.3883, -0.0530],
        [-0.3971, -0.5242, -0.76

tra tensor(nan, grad_fn=<SumBackward0>)
LOSS tensor(nan, grad_fn=<SubBackward0>)
Y tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan

        [nan, nan, nan, nan]], grad_fn=<EluBackward>)
exp tensor(nan, grad_fn=<SumBackward0>)
tra tensor(nan, grad_fn=<SumBackward0>)
LOSS tensor(nan, grad_fn=<SubBackward0>)
Y tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan,

Y tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        

        [nan, nan, nan, nan]], grad_fn=<EluBackward>)
exp tensor(nan, grad_fn=<SumBackward0>)
tra tensor(nan, grad_fn=<SumBackward0>)
LOSS tensor(nan, grad_fn=<SubBackward0>)
Y tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan,

...

Y tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        

Y tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        

        [nan, nan, nan, nan]], grad_fn=<EluBackward>)
exp tensor(nan, grad_fn=<SumBackward0>)
tra tensor(nan, grad_fn=<SumBackward0>)
LOSS tensor(nan, grad_fn=<SubBackward0>)
Y tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan,

Y tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        

Y tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        